In [1]:
import pandas as pd

In [4]:
# 1. Carregar base de dados original
df = pd.read_csv('/content/base_de_dados.csv', encoding='utf-8', low_memory=False)

In [5]:
# 2. Seleção de colunas essenciais para análise
keep_cols = [
    'id', 'companyName', 'category', 'title',
    'description', 'created', 'firstInteractionDate', 'modified',
    'status', 'score', 'evaluation', 'dealAgain', 'hasReply',
    'solved', 'userCity', 'userState'
]
drop_cols = [col for col in df.columns if col not in keep_cols]
df = df.drop(columns=drop_cols)

In [6]:
# 3. Renomear colunas para padrão snake_case e nomenclatura clara
rename_map = {
    'companyName': 'company_name',
    'created': 'created_at',
    'firstInteractionDate': 'first_interaction_at',
    'modified': 'updated_at',
    'userCity': 'user_city',
    'userState': 'user_state',
    'dealAgain': 'deal_again',
    'hasReply': 'has_reply',
}
df = df.rename(columns=rename_map)

In [7]:
# 4. Conversão de colunas de datas para datetime
date_cols = ['created_at', 'first_interaction_at', 'updated_at']
for c in date_cols:
    df[c] = pd.to_datetime(df[c], errors='coerce')

In [8]:
# 5. Cálculo de tempo de resolução (dias entre primeira interação e última atualização)
df['resolution_time_days'] = (df['updated_at'] - df['first_interaction_at']).dt.days


In [9]:
# 6. Limpeza de texto em colunas de título e descrição
df['title'] = df['title'].astype(str).str.strip().str.lower()
df['description'] = df['description'].astype(str).str.strip().str.lower()

In [10]:
# 7. Tratamento de valores ausentes em variáveis categóricas
for col in df.columns:
  df[col] = df[col].fillna('Desconhecido')

In [11]:
# 8. Reordenação das colunas de forma que fiquem semanticamente agrupadas
new_order = [
    'id',
    # Informações da empresa e contexto
    'company_name', 'category',
    # Texto da reclamação
    'title', 'description',
    # Timestamps e cálculo de resolução
    'created_at', 'first_interaction_at', 'updated_at', 'resolution_time_days',
    # Status e avaliação
    'status', 'has_reply', 'solved', 'deal_again', 'score', 'evaluation',
    # Localização do usuário
    'user_city', 'user_state'
]

# Reordenar
df_reordered = df[new_order]

# 8.5 Remover registros duplicados
df_reordered = df_reordered.drop_duplicates()

In [12]:
# 9. Salvando versão limpa para análises futuras
output_path = '/content/base_de_dados_limpo.csv'
df_reordered.to_csv(output_path, index=False, encoding='utf-8')
print(f"Conjunto limpo salvo em: {output_path}")

Conjunto limpo salvo em: /content/base_de_dados_limpo.csv


In [13]:
# 10. Exibir formato final e primeiras linhas
print(f"Formato limpo: {df_reordered.shape}")
df_reordered.head()

Formato limpo: (327, 17)


,id,company_name,category,title,description,created_at,first_interaction_at,updated_at,resolution_time_days,status,has_reply,solved,deal_again,score,evaluation,user_city,user_state
0,86JAtrknAx9xp0NM,Azul,18.0,azul enganação,"fiz uma viagem ***** teresina para manaus, não...",2025-04-27 12:33:14,Desconhecido,2025-04-27 12:33:28,Desconhecido,PENDING,False,False,Desconhecido,Desconhecido,Desconhecido,Manaus,AM
1,RcovFL-mMY1TBgnA,Tommy Hilfiger Brasil - Vestuário,55.0,mochila azul descascasmento e se tornado inuti...,"prezados,<br /><br />gostaria de registrar uma...",2025-04-27 11:32:38,Desconhecido,2025-04-27 11:32:50,Desconhecido,PENDING,False,False,Desconhecido,Desconhecido,Desconhecido,Teresina,PI
2,p-jRh41v7ZCuk7xE,POSTOS CACIQUE,81.0,mal atendimento,queria chamar atenção do proprietário pra seus...,2025-04-27 10:09:58,Desconhecido,2025-04-27 10:10:09,Desconhecido,PENDING,False,False,Desconhecido,Desconhecido,Desconhecido,Teresina,PI
3,GydoWbYpszOAZHx3,GOL,0.0,bagagem,eu comprei uma uma passagem para o piauí e na ...,2025-04-27 06:24:44,Desconhecido,2025-04-27 06:24:55,Desconhecido,PENDING,False,False,Desconhecido,Desconhecido,Desconhecido,São Paulo,SP
4,2aXDHDmE7N_VO5KJ,Lojas Riachuelo e Midway Financeira,3.0,a caixa mentiram para me convencer a contratar...,fiz compras na unidade do shopping teresina. e...,2025-04-26 18:37:47,Desconhecido,2025-04-26 18:37:59,Desconhecido,PENDING,False,False,Desconhecido,Desconhecido,Desconhecido,João Pessoa,PB


# Descrição de cada coluna

- id: Identificador único da reclamação no Reclame Aqui.
- company_name: Nome da empresa contra a qual a reclamação foi registrada.
- category: Código numérico que indica a categoria geral da reclamação.
  - Casos mais comuns:
    - 0 – Não encontrei meu problema
    - 3 – Cartões de Crédito
    - 16 – Ônibus Rodoviário
    - 18 – Companhias Aéreas
    - 33 – Planos de Saúde
    - 43 – Salões e Centros de Estética
    - 55 – Bolsas e Malas
    - 118 – Logística e Entrega Rápida
    - 233 – Moda Feminina
    - 255 – Problemas com o Atendimento
    - 260 – Problemas na Loja
- title: Título livre da reclamação.
- description: Texto completo do relato do consumidor.
- created_at: Data e hora em que o consumidor postou originalmente a reclamação.
- first_interaction_at: Data e hora do primeiro registro de interação subsequente (pode ser a primeira resposta da empresa ou a moderação).
- updated_at: Data e hora da última atualização na thread da reclamação (nova resposta, avaliação, moderação, etc.).
- status: Estado atual da reclamação no sistema (“PENDING”, “IN_PROGRESS”, “SOLVED”, etc.).
- score: Nota numérica (geralmente de 1 a 5) atribuída pelo consumidor ao atendimento recebido.
- evaluation: Texto livre de avaliação/pós-atendimento deixado pelo consumidor (quando disponível).
- deal_again: Booleano (True/False) indicando se o consumidor responderia “sim” ou “não” à pergunta “Você negociaria com esta empresa novamente?”.
- has_reply: Booleano indicando se a empresa chegou a responder à reclamação.
- solved: Booleano que marca se a reclamação foi considerada solucionada pelo consumidor ou pela empresa.
- user_city: Cidade de origem do reclamante.
- user_state: Estado de origem do reclamante.
- resolution_time_days: Tempo em dias entre a first_interaction_at e a updated_at, uma métrica de velocidade/resolução do atendimento.

# Documentação do processamento da base de dados.

1. Carregamento e inspeção inicial

O dataset original tinha 100 registros e 68 colunas, muitas delas sem relevância para análise.

2. Seleção de colunas essenciais

Mantivemos apenas 17 colunas: identificador (id), informações da empresa, categoria, texto da reclamação (título e descrição), datas-chave, status e métricas de avaliação, além de localização (cidade/estado) e flags como dealAgain, hasReply e solved.

3. Padronização de nomes

Renomeamos as colunas para snake_case e termos mais claros (por exemplo, created → created_at, firstInteractionDate → first_interaction_at, etc.).

4. Conversão de datas

Transformamos colunas de string para datetime, validando erros de formatação(parsing).

5. Criação de nova variável “tempo de resolução”

resolution_time_days = diferença em dias entre a primeira interação e a última atualização.

6. Limpeza de texto

Títulos e descrições foram convertidos para minúsculas, com remoção de espaços extremos.

7. Tratamento de valores ausentes

Em todas as colunas, valores nulos(NaN) foram renomeados com 'Desconhecido'.

8. Reordenação das colunas de forma que fique agrupada de forma lógica